Classification Model

In [1]:
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, confusion_matrix

In [2]:
df_pluviometricos = pd.read_csv("../dados/dados_pluviometricos_geo_anos.csv")

In [3]:
df_curva_niveis = pd.read_csv("../dados/dados_curva_niveis.csv")

In [4]:
df_curva_niveis = df_curva_niveis[["score", "hex", "geometry", "Elevation"]]

In [5]:
df_curva_niveis = df_curva_niveis.groupby(["hex", "geometry"], as_index=False)["Elevation", "score"].mean()

/var/folders/ld/8dhf4gy93xbd1ybmf6nljm38dn1p1r/T/ipykernel_27314/975208766.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_curva_niveis = df_curva_niveis.groupby(["hex", "geometry"], as_index=False)["Elevation", "score"].mean()


Preparar dados de Lagos, Rios e Cavas

In [6]:
df_hidro = pd.read_csv("../dados/lagos_rios.csv")

Preparar Feature do Solo

In [32]:
df_feature_solo = pd.read_csv("../dados/feature_solo.csv")

Preparar o dataset de Feature de Uso de Solos

In [35]:
df_feature_uso_solo = pd.read_csv("../dados/feature_uso_solos.csv")

Preparar o dataset de alagamentos (o target)

In [7]:
df_alagamentos = pd.read_csv("../dados/alagamentos.csv")

In [8]:
df_alagamentos["flooded"] = df_alagamentos["NATUREZA"]

In [9]:
df_alagamentos["date"] = df_alagamentos["DATA"]
df_alagamentos = df_alagamentos[["flooded", "date", "hex", "NOME", "geometry"]]

In [10]:

df_alagamentos.date = df_alagamentos.date.astype('datetime64[ns]')

/var/folders/ld/8dhf4gy93xbd1ybmf6nljm38dn1p1r/T/ipykernel_27314/3419792466.py:1: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df_alagamentos.date = df_alagamentos.date.astype('datetime64[ns]')


In [11]:
df_alagamentos['date'] = df_alagamentos['date'].dt.strftime('%Y-%m-%d')

Preparar a elevação

In [12]:
df_merged = pd.merge(df_alagamentos, df_curva_niveis, on="geometry", how="outer")

In [13]:
df_merged["elevation"] = df_merged["Elevation"]
df_merged["hex"] = df_merged["hex_x"]
df_merged["score_elevation"] = df_merged["score"]
df_merged = df_merged[["flooded", "date", "hex", "geometry", "score_elevation", "elevation"]]

In [14]:
df_merged.drop_duplicates(inplace=True)

In [15]:
df_merged = pd.merge(df_pluviometricos, df_merged, on=["geometry", "date"], how="left")

In [16]:
df_merged = pd.merge(df_merged, df_curva_niveis, on=["geometry"], how="left")

In [17]:

df_merged["value_precipitation"] = df_merged["valorMedido"]
df_merged["value_precipitation_3_days"] = df_merged["value_last_3_days"]
df_merged["value_precipitation_7_days"] = df_merged["value_last_7_days"]
df_merged["hex"] = df_merged["hex_x"]
df_merged = df_merged[["flooded", "date", "hex", "geometry", "score_elevation", "elevation", "value_precipitation", "value_precipitation_3_days", "value_precipitation_7_days"]]

In [18]:
df_merged = pd.merge(df_merged, df_curva_niveis[['geometry', 'score', 'Elevation']], on='geometry', how='left')

In [19]:
df_merged['score_elevation'].fillna(df_merged['score'], inplace=True)
df_merged['elevation'].fillna(df_merged['Elevation'], inplace=True)
df_merged['flooded'].fillna("Nenhum", inplace=True)

In [20]:
df_merged.drop(columns='score', inplace=True)
df_merged.drop(columns='Elevation', inplace=True)

In [21]:
df_merged = pd.merge(df_merged, df_hidro[['hex', 'score_hidro', 'neighbors_hidro']], on='hex', how='left')

In [22]:
df_merged["score_hidro"].fillna(0.0, inplace=True)
df_merged["neighbors_hidro"].fillna(0.0, inplace=True)

In [33]:
df_merged = pd.merge(df_merged, df_feature_solo, on=['hex', "geometry"], how='left')

In [36]:
df_merged = pd.merge(df_merged, df_feature_uso_solo, on=["hex", "geometry"], how="left")

In [37]:
df_merged.head()

,flooded,date,hex,geometry,score_elevation,elevation,value_precipitation,value_precipitation_3_days,value_precipitation_7_days,score_hidro,...,VEGETACAO,MECANIZACA,PRIM_NIVEL,Unnamed: 0_y,SPRAREA,SPRPERIMET,SPRCLASSE,DESCLASSE,PERCLASSE,GRCLASSE
0,Nenhum,2015-01-01,88a804cb05fffff,POLYGON ((-49.32705987407714 -25.3834767821367...,99.5,980.651163,11.598242,0.0,0.0,0.0,...,fl. subtrop. peren.,M2,Latossolo,718,4.643871e+08,2.213961e+05,Area urbana,área urbana,100%,Área urbana
1,Nenhum,2015-01-01,88a804cb05fffff,POLYGON ((-49.32705987407714 -25.3834767821367...,99.5,980.651163,11.598242,0.0,0.0,0.0,...,fl. subtrop. peren.,M2,Latossolo,719,4.643871e+08,2.213961e+05,Area urbana,área urbana,100%,Área urbana
2,Nenhum,2015-01-01,88a804cb05fffff,POLYGON ((-49.32705987407714 -25.3834767821367...,99.5,980.651163,11.598242,0.0,0.0,0.0,...,fl. subtrop. peren.,M2,Latossolo,718,4.695525e+09,1.095549e+06,VS+AC,vegetação secundária + agricultura cíclica,65% VS e 35% AC,Vegetação secundária
3,Nenhum,2015-01-01,88a804cb05fffff,POLYGON ((-49.32705987407714 -25.3834767821367...,99.5,980.651163,11.598242,0.0,0.0,0.0,...,fl. subtrop. peren.,M2,Latossolo,719,4.695525e+09,1.095549e+06,VS+AC,vegetação secundária + agricultura cíclica,65% VS e 35% AC,Vegetação secundária
4,Nenhum,2015-01-01,88a804cb05fffff,POLYGON ((-49.32705987407714 -25.3834767821367...,99.5,980.651163,11.598242,0.0,0.0,0.0,...,fl. subtrop. peren.,M2,Latossolo,718,4.643871e+08,2.213961e+05,Area urbana,área urbana,100%,Área urbana


In [23]:
df_merged['date'] = pd.to_datetime(df_merged['date'])
df_merged['year'] = df_merged['date'].dt.year
df_merged['month'] = df_merged['date'].dt.month
df_merged['day'] = df_merged['date'].dt.day

In [24]:
df_merged.drop_duplicates(inplace=True)

In [25]:
df_merged[df_merged["flooded"] == "Alagamento"]

,flooded,date,hex,geometry,score_elevation,elevation,value_precipitation,value_precipitation_3_days,value_precipitation_7_days,score_hidro,neighbors_hidro,year,month,day
862896,Alagamento,2015-10-22,88a804cb41fffff,POLYGON ((-49.33694002968239 -25.4052233804035...,99.5,967.049180,60.544994,52.215987,54.987237,100.0,0.0,2015,10,22
862900,Alagamento,2015-10-22,88a804cb41fffff,POLYGON ((-49.33694002968239 -25.4052233804035...,99.5,967.049180,60.544994,52.215987,54.987237,99.0,0.0,2015,10,22
924221,Alagamento,2015-10-21,88a804cb49fffff,POLYGON ((-49.34023347844297 -25.4124704980994...,99.5,953.672131,59.109061,0.000000,1.403529,0.0,0.0,2015,10,21
3129076,Alagamento,2015-08-02,88a804d86dfffff,POLYGON ((-49.355572751255856 -25.503956849950...,96.0,920.290698,0.000000,0.000000,0.294974,96.0,103424.0,2015,8,2
3161950,Alagamento,2017-12-22,88a804d905fffff,POLYGON ((-49.309048293845166 -25.478700437163...,100.0,915.345455,41.254045,9.868296,25.355981,0.0,0.0,2017,12,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8451853,Alagamento,2016-08-20,88a831adb1fffff,POLYGON ((-49.2122644072129 -25.49699169415437...,99.5,892.905660,56.589717,51.774454,63.194167,0.0,0.0,2016,8,20
8452962,Alagamento,2016-09-02,88a831adbbfffff,POLYGON ((-49.2214295394814 -25.49788388919803...,99.5,878.187500,0.204890,12.220728,12.285600,0.0,0.0,2016,9,2
9951242,Alagamento,2016-10-20,88a831ae95fffff,POLYGON ((-49.3119299050174 -25.56199807088341...,98.0,920.306306,29.058548,81.846152,123.845441,0.0,0.0,2016,10,20
10916412,Alagamento,2015-09-10,88a831ae9bfffff,POLYGON ((-49.330254284209296 -25.563764979069...,99.5,893.704918,19.291447,49.208366,74.716444,0.0,0.0,2015,9,10


Rodar modelo de classificação

In [ ]:
import pandas as pd
import geopandas as gpd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

features = ['elevation', 'score_elevation', 'value_precipitation', 'value_precipitation_3_days', 'value_precipitation_7_days', "date"]
target = 'flooded'

# Dividir os dados em conjunto de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(df_merged[features], df_merged[target], test_size=0.2, random_state=42)

# Criar e treinar a árvore de decisão
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)

# Avaliar o desempenho do modelo no conjunto de teste
score = clf.score(X_test, y_test)
print("Acurácia do modelo:", score)


In [26]:
features_elevation = ['elevation', 'score_elevation']
features_precipitation = ['value_precipitation', 'value_precipitation_3_days', 'value_precipitation_7_days']
features_hidro = ['score_hidro', 'neighbors_hidro']
date = ["year", "month", "day"]

In [27]:
features = df_merged[features_elevation+features_precipitation+features_hidro+date]
target = df_merged[['flooded']]

In [28]:
train_features, test_features, train_labels, test_labels = train_test_split(features, target, test_size=0.2)

In [29]:
rf = RandomForestClassifier()
rf.fit(train_features, train_labels)

/var/folders/ld/8dhf4gy93xbd1ybmf6nljm38dn1p1r/T/ipykernel_27314/2852299628.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(train_features, train_labels)


RandomForestClassifier()

In [30]:
predictions = rf.predict(test_features)

In [31]:
print("Accuracy: " + str(accuracy_score(test_labels, predictions)))
print("Recall for nothing: " + str(recall_score(test_labels, predictions, average="binary", pos_label="Nenhum")))
print("Recall for flooded: " + str(recall_score(test_labels, predictions, average="binary", pos_label="Alagamento")))
print(confusion_matrix(test_labels, predictions))

Accuracy: 0.9996578467153284
Recall for nothing: 1.0
Recall for flooded: 0.0
[[    0    33]
 [    0 96415]]


In [ ]:
print("Len: " + str(len(predictions)))
alagamentos = []
nenhum = []
for f in predictions:
    if f == "Nenhum":
        nenhum.append(f)
    if f == "Alagamento":
        alagamentos.append(f)

print("Len alagamentos: " + str(len(alagamentos)))
print("Len nenhum: " + str(len(nenhum)))

Verificar a porcentagem de flooded

In [ ]:

total_alagamentos = len(df_merged[df_merged['flooded'] == "Alagamento"])
total_nenhum = len(df_merged[df_merged['flooded'] == "Nenhum"])

print("Alagamentos: " + str(total_alagamentos))
print("Nenhum: " + str(total_nenhum))
print("Porcentagem: " + str(100*total_alagamentos/(total_alagamentos+total_nenhum)) + "%")